In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
from scipy.optimize import minimize
from cosamp import cosamp

plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams.update({'font.size': 18})

## Signal generation

In [2]:
n = 4096 # sample rate of high resolution signal
t = np.linspace(0, 1, n)
f1 = 100
f2 = 800
x = np.cos(2 * np.pi * f1 * t) + np.cos(2 * np.pi * f1 * t)
x_fft = np.fft.fft(x)
x_PSD = x_fft * np.conj(x_fft) / n

## random sampling

In [3]:
p = 128 # number of random samples
perm = np.floor(np.random.rand(p) * n).astype(int)
y = x[perm]

## plot time- and PSD-signal

In [ ]:
time_window = np.array([1024, 1280]) / n
freq = np.arange(n)
L = int(np.floor(n/2))

fig, axs = plt.subplots(1, 2)

axs[0].plot(t, x, LineWidth=2)
axs[0].plot(perm/n, y, marker='o', LineWidth=0, ms=12, mew=4)
axs[0].set_xlim(time_window[0], time_window[2])
axs[0].set_ylim(-2, 2)
axs[0].set_xlabel('Time [s]')

axs[1].plot(freq[:L], x_PSD[:L], LineWidth=2)
axs[1].set_xlim(0, 1024)
axs[1].set_xlim(0, 1200)
axs[1].set_xlabel('Frequency [Hz]')

plt.show()